---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def remove_to_end(s, character):
    if not isinstance(s, str):
        return s
    if character in s:
        start = s.find(character)
        return s[:start]
    return s

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    raw = pd.read_table('university_towns.txt', header=None)
    raw.columns = ['raw']
    raw['is_state'] = raw.iloc[:,0].str.contains('edit')
    raw['State'] = raw.loc[raw['is_state'], 'raw'].str.strip()
    raw['State'] = raw['State'].map(lambda x: remove_to_end(x, '['))
    raw['State'].fillna(method='ffill', inplace=True)
    raw.drop(raw.loc[raw['is_state']].index, inplace=True)
    raw['RegionName'] = raw['raw'].map(lambda x: remove_to_end(x, ' ('))
    return raw[['State', 'RegionName']]
%timeit get_list_of_university_towns()

In [ ]:
def get_year(s):
    return int(s[:4])

gdp = pd.read_excel('gdplev.xls')
gdp_cols = ['qy', 'drop', 'current']
gdp = gdp.iloc[:, 4:7].dropna()
gdp.columns = gdp_cols
gdp.drop('drop', axis=1, inplace=True)
gdp['year'] = gdp['qy'].map(get_year)
gdp = gdp[['year', 'qy', 'current']]
gdp = gdp.loc[gdp['year']>= 2000]
gdp.reset_index(drop=True, inplace=True)
gdp['previous'] = gdp['current'].shift(1)
gdp['increase'] = gdp['current'] > gdp['previous']
gdp.loc[0, 'increase'] = np.nan
gdp['increase_prev'] = gdp['increase'].shift(1)
gdp['recesion'] = (gdp['increase_prev'] + gdp['increase']) == 0
gdp['recesion_end'] = (gdp['increase_prev'] + gdp['increase']) == 2

def get_recession_start():
    rec_start = gdp.loc[gdp['recesion']].iloc[0].name - 1
    return gdp.loc[rec_start, 'qy']
%timeit get_recession_start()

In [ ]:
def get_recession_end():
    rec_start = gdp.loc[gdp['recesion']].iloc[0].name - 1
    gdp_rec = gdp.loc[rec_start:]
    rec_end = gdp_rec.loc[gdp['recesion_end']].iloc[0].name
    return gdp.loc[rec_end, 'qy']
%timeit get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    rec_start = gdp.loc[gdp['recesion']].iloc[0].name - 1
    gdp_rec = gdp.loc[rec_start:]
    rec_end = gdp_rec.loc[gdp['recesion_end']].iloc[0].name
    gdp_rec = gdp.loc[rec_start: rec_end]
    return gdp_rec.loc[gdp_rec['current'] == gdp_rec['current'].min(), 'qy'].iloc[0]
%timeit get_recession_bottom()

In [ ]:
def convert_to_quarter_and_year(s):
    dt = pd.to_datetime(s)
    year = dt.year
    quarter = dt.quarter
    return str(year)+'q'+str(quarter)
    
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    cols = df.columns.tolist()
    new_cols = cols[:6] + [convert_to_quarter_and_year(c) for c in cols[6:]]
    df.columns = new_cols
    new_df = df[new_cols[:6]]
    used_q = []
    for q in new_cols[6:]:
        if q not in used_q and (2000<=int(q[:4])<=2015 or int(q[:4])==2016 and int(q[-1]) != 4):
            new_df[q] = df[q].mean(axis=1)
        used_q.append(q)
    new_df.drop(['Metro', 'CountyName', 'SizeRank', 'RegionID'], axis=1, inplace=True)
    new_df['State'] = new_df['State'].replace(states)
    return new_df.set_index(['State', 'RegionName'])
convert_housing_data_to_quarters()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    all_data = convert_housing_data_to_quarters()
    university_towns = get_list_of_university_towns() 
    start = get_recession_start()
    end = get_recession_bottom()
    cols = []
    flag = False
    for c in all_data.columns:

        if c == start:
            flag = True
        if flag:
            cols.append(c)
        if c == end:
            flag = False
            
    all_data = all_data[cols]
    merged_data = all_data.merge(university_towns, left_index=True, right_on=['State', 'RegionName'], how='left', indicator=True)
    
    uni_data = merged_data.loc[merged_data['_merge'] == 'both'].drop(['_merge', 'State', 'RegionName'], axis=1)
    other_data = merged_data.loc[merged_data['_merge'] == 'left_only'].drop(['_merge', 'State', 'RegionName'], axis=1)
    statistic, pvalue = ttest_ind(other_data.mean(), uni_data.mean())
    uni_pr = uni_data.mean()[end]/uni_data.mean()[start]
    other_pr = other_data.mean()[end]/other_data.mean()[start]
    return pvalue < 0.01, pvalue, "university town" if uni_pr > other_pr else  "non-university town"
%timeit run_ttest()